In [1]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=26cc475178fe92d868d02b41b8951827e18db3c4c136058637527c8310275789
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# Dataset

In [3]:
import os

root = "/kaggle/input/"
def read_data(root, folder):
    data = {'en':[], 'vi':[]}
    path = os.path.join(root, folder)
    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        with open(file_path,'r') as f:
            _, tail = file_path.split('.')
            if tail =='en':
                for line in f:
                    data['en'].append(line.strip())
            else:
                for line in f:
                    data['vi'].append(line.strip())
                    
    return data

In [4]:
root = "/kaggle/input/"

train_data = read_data(root, 'trainnew')
test_data = read_data(root, 'testnew')
valid_data = read_data(root, 'validd')

In [5]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
valid_dataset = Dataset.from_dict(valid_data)

In [6]:
from datasets import DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "valid": valid_dataset
})

# Model

In [7]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer for Vietnamese to English
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
def translate_en_to_vi(sentence):
    """
    Translate a list of English texts into Vietnamese.
    Args:
        texts (list of str): The English texts to translate.
    Returns:
        list of str: Translated Vietnamese texts.
    """
    # Tokenize the input text
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    
    # Generate translations
    translated = model.generate(**inputs)

    translated_sentence = tokenizer.decode(translated[0], skip_special_tokens=True)
    
    # Decode the output to human-readable text
    return translated_sentence

In [9]:
# Example English sentences
english_texts = "Hello, how are you?"

# Translate to Vietnamese
vietnamese_translations = translate_en_to_vi(english_texts)
print(vietnamese_translations)


Xin chào, anh khỏe không?


In [10]:
final_test = read_data(root, 'final-datatest')

In [11]:
final_datatest = Dataset.from_dict(final_test)
final_datatest

Dataset({
    features: ['en', 'vi'],
    num_rows: 100
})

In [12]:
import torch

# Check if CUDA is available and move the model to GPU if possible
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [13]:
def translate(dataset):
    texts = []
    predictions = []
    references = []
    for item in dataset:
        source = item['en']
        target = item['vi']

        inputs = tokenizer(source, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)

        prediction = tokenizer.decode(translated[0], skip_special_tokens=True)

        texts.append(source)
        predictions.append(prediction)
        references.append(target)


    return texts, predictions, references

In [14]:
texts, predictions, references = translate(final_datatest)

In [15]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu_scores(predictions, references, weights = (1,0,0,0)):

    smoothing = SmoothingFunction().method1
    BLEU_scores = []
    
    for pred, ref in zip(predictions, references):
        ref_tokens = ref.split()
        pred_tokens = pred.split()

        bleu = sentence_bleu([ref_tokens], pred_tokens, weights=weights, smoothing_function=smoothing)
        BLEU_scores.append(bleu)
    
    return BLEU_scores

In [16]:
scores = calculate_bleu_scores(predictions, references, weights = (1,0,0,0))
scores1 = calculate_bleu_scores(predictions, references, weights = (0.5,0.5,0,0))

In [17]:
import pandas as pd

data ={
         'texts': texts,
         'predictions': predictions,
         'references': references,
         'BLEU_1': scores,
         'BLEU_2': scores1
     }
df = pd.DataFrame(data)

In [18]:
df

,texts,predictions,references,BLEU_1,BLEU_2
0,Murillo has served as the Nicaraguan governmen...,Lời thề đã trở thành thủ lĩnh của chính phủ ái...,Murillo đã từng là phát ngôn viên chính của ch...,0.360891,0.258525
1,Work : People may regret not following a diffe...,Công việc: Người ta có thể hối tiếc vì không t...,Công việc : Con người có thể cảm thấy hối tiếc...,0.432775,0.343115
2,M- my wife barely lets me see her naked .,Vợ tôi không cho tôi thấy cô ấy khỏa thân.,Vợ ... vợ tôi còn hiếm khi để tôi nhìn cô ấy k...,0.303265,0.142961
3,"It 's full of panic and fear , and I 'd heard ...","Đó là đầy đủ của hoảng loạn và sợ hãi, và tôi'...",Nó chứa đầy sợ hãi và hoảng loạn và tôi nghe t...,0.521216,0.308006
4,This will make it much easier for you to expos...,Điều này sẽ giúp bạn dễ dàng hơn nhiều trong v...,Bước này sẽ giúp bạn dễ dàng làm lộ rễ cây để ...,0.342857,0.265684
...,...,...,...,...,...
95,You can click on any text box and start typing...,Bạn có thể nhắp vào hộp văn bản nào và bắt đầu...,Bạn có thể nhấp chuột vào bất kỳ hộp thoại nào...,0.577919,0.418742
96,"Yeah , they 're rocket scientists , remember ?","Phải, họ là nhà khoa học, nhớ không?","Ừ , họ là các nhà khoa học tên lửa , nhớ không ?",0.295229,0.199611
97,Fidgeting is a sign that you lack confidence .,Đánh lừa là dấu hiệu cho thấy bạn thiếu tự tin.,Đó là dấu hiệu cho thấy sự thiếu tự tin của bạn .,0.606366,0.502772
98,Agreeing with the angry person might help diff...,Đồng ý với người hay giận có thể giúp người ấy...,Việc bày tỏ sự đồng tình với người đang tức gi...,0.142575,0.074198


In [19]:
df['BLEU_1'].sum()/len(df)

0.393201261632694

# Finetune

## Tokenizer_dataset

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 297799
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['en', 'vi'],
        num_rows: 4679
    })
})

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 297799
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['en', 'vi'],
        num_rows: 4679
    })
})

In [22]:
def preprocess(example):
    inputs = tokenizer(example["en"], padding="max_length", truncation=True, max_length=64)
    outputs = tokenizer(text_target = example["vi"], padding="max_length", truncation=True, max_length=64)

    example["input_ids"] = inputs.input_ids
    example["attention_mask"] = inputs.attention_mask
    example["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in outputs.input_ids
    ]
    return example

In [23]:
batch_size=128

tokenizer_dataset = dataset.map(
    preprocess, 
    batched=True, 
    batch_size=batch_size,
    remove_columns=["en", "vi"]
)

Map:   0%|          | 0/297799 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/4679 [00:00<?, ? examples/s]

In [24]:
# Load your parallel dataset
train_dataset = tokenizer_dataset['train']
val_dataset = tokenizer_dataset['valid']

## Training

In [25]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs = 5,  # Set a maximum number of epochs
    save_total_limit=2,
    predict_with_generate=True,
    report_to="none",
    metric_for_best_model="eval_loss",  # Specify the metric to monitor
    greater_is_better=False,  # Smaller loss is better
    load_best_model_at_end=True,  # Load the best model at the end of training
    save_strategy="epoch"  # Save the model at the end of each epoch
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2  # Stop training if no improvement for 3 consecutive evaluations
)

In [27]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # Add the early stopping callback
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.636500,1.429389
2,1.486200,1.390964
3,1.400000,1.375704
4,1.336000,1.368785
5,1.287100,1.367624


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/par

TrainOutput(global_step=11635, training_loss=1.441726638669357, metrics={'train_runtime': 9174.0655, 'train_samples_per_second': 162.305, 'train_steps_per_second': 1.268, 'total_flos': 2.523724794298368e+16, 'train_loss': 1.441726638669357, 'epoch': 5.0})